# Download and import packages

In [ ]:
import wrds
import pandas as pd
import calendar
import numpy as np
import time
import unicodedata
import re
from functools import partial
from nltk.corpus import stopwords
import textstat
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer(language='english')
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from multiprocessing import Pool
from nltk.sentiment.util import mark_negation
import docx
from docx.shared import Pt
import warnings
import datetime
import sys
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [ ]:
# Get the list of company ids that we are interested in
gvkey = pd.read_csv('gvkey.csv')
gvkey_companyid = pd.merge(gvkey, ID_Mapping_Table[['companyid', 'gvkey']], on='gvkey', how='left')
companyid_list = gvkey_companyid[gvkey_companyid['companyid'].notna()]['companyid'].astype(int).tolist()

# Extract 10-year earnings call data and save it

In [ ]:
def load_all_earnings_calls(db,year,month):
    '''
    This function returns all call IDs that occurred in a given time interval.  Note the
    passed in dates are interpreted as UTC dates, which doesn't matter much because the
    month level aggregation happens later, and will be done by NY time presumably.

    year, month -- numerical value of month to load
    '''
    def status(objin):
        print('\tRows in month: {:,}'.format(objin.shape[0]))


    # select all id's that appear in this month, as well as any records with these id's in other
    # months; the mostimportantdateutc field is the date of the call, but calls often get revised
    # in the following months

    relevant_columns = '''companyid, keydevid, transcriptid, headline, keydeveventtypename,
                          transcriptcollectiontypeid, transcriptcreationdate_utc'''
    sql_query = '''
    select {} from ciq.wrds_transcript_detail
    where mostimportantdateutc between '{}-{:02d}-{:02d}' and '{}-{:02d}-{}'
    '''.format(relevant_columns, year,month,1,year,month,calendar.monthrange(year,month)[1])

    print('Doing fetch',sql_query)
    calls = db.raw_sql(sql_query)

    # select the right type of calls
    calls = calls[calls.keydeveventtypename == 'Earnings Calls']
    print('\tFound {:,} calls'.format(len(calls.groupby('keydevid'))))
    status(calls)

    # save all the companyIDs of a transcript into a list, one transcript may mention two or more companies
    # drop for a while, merge back at the end of the whole process
    calls.companyid = calls.companyid.apply(lambda x :int(x))
    companyIDs = calls.groupby('keydevid')['companyid'].unique().reset_index()
    calls.drop('companyid', axis=1, inplace=True)

    # now select the most recent observation and the highest transcriptcollectiontypeid
    def select_type(obj):
        # take the last observations
        last_date = obj.transcriptcreationdate_utc.unique().max()

        # take the largest available transcriptcollectiontypeid (allowable values are 1,2,7,8)
        # Select with priority from high to low as:
        # 8 (audited copy), 1 (proofed copy), 2 (edited copy), 7 (spell chekced copy)
        return(obj[obj.transcriptcreationdate_utc==last_date].sort_values('transcriptcollectiontypeid').\
               iloc[0])

    # select the correct version of each transcript
    calls = calls[calls['transcriptcollectiontypeid'].isin([1,2,7,8])]
    calls.transcriptcollectiontypeid=calls.transcriptcollectiontypeid.replace({8:1,1:2,2:3,7:4})
    calls = calls.groupby('keydevid').apply(select_type).reset_index(drop=True)
    status(calls)

    # Get the detailed time in the original event date from ciqkeydev db
    # mostimportantdateutc in the wrds_transcript_detail db does not provide specific time in a day
    sql_query2 = '''
    select keydevid, mostimportantdateutc from ciq.ciqkeydev where keydevid in
    (select keydevid from ciq.wrds_transcript_detail where
    mostimportantdateutc between '{}-{:02d}-{:02d}' and '{}-{:02d}-{}')
    '''.format(year,month,1,year,month,calendar.monthrange(year,month)[1])

    call_detail_time = db.raw_sql(sql_query2)
    calls = calls.merge(call_detail_time, on='keydevid', how='left')


    # for the selected calls, get the transcripts, and relevant information

    sql_query3 = '''
    SELECT transcriptid, componenttext, componentorder, transcriptcomponentid, transcriptcomponenttypeid, transcriptpersonid
    FROM ciq_transcripts.ciqtranscriptcomponent
    WHERE transcriptid in ({})
    ORDER BY transcriptid, componentorder
    '''.format(','.join(['{:.0f}'.format(el) for el in calls.transcriptid.unique()]))

    print('Selecting components from {:,} transcripts'.format(len(calls.transcriptid.unique())))
    comps = db.raw_sql(sql_query3)


    # Get Table for all personname
    sql_query4 = '''
    SELECT transcriptpersonid, transcriptpersonname
    FROM ciq_transcripts.ciqtranscriptperson
    '''
    speaker_info = db.raw_sql(sql_query4)
    speaker_info['transcriptpersonname_id'] = speaker_info['transcriptpersonname']\
        +speaker_info['transcriptpersonid'].apply(lambda x:'_<id: '+str(int(x))+'>')
    comps = pd.merge(comps,speaker_info,on='transcriptpersonid',how='left')
    comps['transcriptpersonname_id'] = comps['transcriptpersonname_id'].fillna('Unknown Person')

    # Now recover the presentation and Q&A sections
    # Break one transcript into blocks, for Presentations, divide into paragraphs ,
    # for QA, divide into question-answer pairs (Q and folloing As until next Q)
    # the presentation section has transcriptcomponenttypeid == 2, and the Q&A have 3&4 respectively;
    # NOTE: we combine the Q&A into one block of text, and do not differentiate between them
    def break_into_blocks(obj):
        obj = obj.sort_values(['transcriptid','componentorder'])

        # Presentations
        pre = obj[obj.transcriptcomponenttypeid.isin([2])]

        # Change the punctuations to split the transcripts into paragraphs
        pre['componenttext'] = pre['componenttext'].apply(lambda x: x.replace('\r\n', '\n').replace('\r', '\n').split('\n'))
        pre = pre.explode(column='componenttext')

        # In subsequent analysis, we use keydevid_transcriptid_section_blocknum as an identifier for each block
        pre['block_num'] = pre.groupby('transcriptid')['componenttext'].cumcount()
        pre['section'] = 'Pre'
        pre['pretty_text'] = '[Presentation] '+ pre['transcriptpersonname_id'] + '\n' + pre['componenttext']

        # QA
        qa = obj[obj.transcriptcomponenttypeid.isin([3,4])]
        qa['Ques'] = qa['transcriptcomponenttypeid']==3
        qa['transcriptpersonname_id'] = np.where(qa['Ques'], 'Q--' + qa['transcriptpersonname_id'].astype(str),'A--' + qa['transcriptpersonname_id'].astype(str))
        qa['block_num'] = qa['Ques'].cumsum() # cumsum only increases if there is a new question
        qa['pretty_text'] = qa['transcriptpersonname_id'] + '\n' + qa['componenttext']
        qablocks = qa.groupby(['block_num','transcriptid']).apply(lambda x: '\n'.join(x['componenttext'])).reset_index().rename(columns={0:'QA'})
        qablocks_pretty = qa.groupby(['block_num','transcriptid']).apply(lambda x: '\n'.join(x['pretty_text'])).reset_index().rename(columns={0:'QA_pretty'})

        qa = pd.merge(qa, qablocks, on=['block_num', 'transcriptid'], how='left') # merge the combined QA to original df
        qa = pd.merge(qa, qablocks_pretty, on=['block_num', 'transcriptid'], how='left')
        qa['componenttext'] = qa['QA']
        qa['pretty_text'] = '[Q&A] ' + qa['QA_pretty']

        qa = qa.sort_values(by=['transcriptid', 'componentorder'])
        qa = qa.drop_duplicates(['block_num','transcriptid'], keep='first')

        # In subsequent analysis, we use keydevid_transcriptid_section_blocknum as an identifier for each block
        qa['block_num'] = qa.groupby('transcriptid')['block_num'].cumcount()
        qa['section'] = 'QA'

        # return relevant columns
        relev_cols = ['transcriptid', 'componenttext', 'block_num', 'section','pretty_text']
        return pd.concat([pre[relev_cols], qa[relev_cols]])

    comps = break_into_blocks(comps)

    # drop irrelevant columns, merge companyids back and change variable types
    drop_columns = ['transcriptcreationdate_utc', 'transcriptcollectiontypeid', 'keydeveventtypename']
    calls = calls.drop(drop_columns, axis=1)
    calls = calls.merge(companyIDs, on='keydevid')
    int_columns = ['keydevid', 'transcriptid']
    calls[int_columns] = calls[int_columns].astype(int)
    calls = calls.merge(comps,on='transcriptid')

    # deal with transcripts with multiple companyids
    calls = calls.reset_index(drop=True)
    calls['companyid']=calls['companyid'].apply(lambda x:re.sub('\[|\]','',str(x)).strip().split(' '))

    # return the merged data
    calls['Id'] = calls.apply(lambda x:str(x['keydevid'])+'_'+str(x['transcriptid'])+'_'+
                                     str(x['section'])+'_'+str(x['block_num']), axis=1)
    return calls


In [ ]:
def save_ec_data(year):

    """
    Fetch yearly earnings calls and extract Pres and QA blocks.
    Separate NA and nonNA fiirms.
    """

    start_year = year
    end_year = year
    start_month = 1
    end_month = 12

    monthly_df = []

    # Loop through the months
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == start_year and month < start_month:
                continue
            if year == end_year and month > end_month:
                break

            db = wrds.Connection(wrds_username='pengfei_ren')
            try:
                print(f'Fetching ECs from {year}-{month:02d} ...')
                data = load_all_earnings_calls(db, year, month)

                data['companyid'] = data['companyid'].apply(lambda x: x[0] if isinstance(x, list) and x else None)
                data['companyid'] = data['companyid'].astype(int)
                filtered_data = data[data['companyid'].isin(companyid_list)]
                monthly_df.append(filtered_data)
            except Exception as e:
                print(f'No data in {year}-{month:02d}: {e}')
            finally:
                db.close()

    # Combine all the monthly data into one DataFrame
    full_period_df = pd.concat(monthly_df)

    return full_period_df

In [ ]:
for year in range(2015, 2024):
    annual_ec_data = save_ec_data(year)
    save_path = f'D:/EC_data/{year}_ec_data.parquet'
    annual_ec_data.to_parquet(save_path, index=False)

Loading library list...
Done
Fetching ECs from 2015-01 ...
Doing fetch 
    select companyid, keydevid, transcriptid, headline, keydeveventtypename, 
                          transcriptcollectiontypeid, transcriptcreationdate_utc from ciq.wrds_transcript_detail 
    where mostimportantdateutc between '2015-01-01' and '2015-01-31'
    
	Found 824 calls
	Rows in month: 2,857
	Rows in month: 824
Selecting components from 824 transcripts
Loading library list...
Done
Fetching ECs from 2015-02 ...
Doing fetch 
    select companyid, keydevid, transcriptid, headline, keydeveventtypename, 
                          transcriptcollectiontypeid, transcriptcreationdate_utc from ciq.wrds_transcript_detail 
    where mostimportantdateutc between '2015-02-01' and '2015-02-28'
    
	Found 2,636 calls
	Rows in month: 9,111
	Rows in month: 2,636
Selecting components from 2,636 transcripts
Loading library list...
Done
Fetching ECs from 2015-03 ...
Doing fetch 
    select companyid, keydevid, transcriptid

In [ ]:
for year in range(2024, 2025):
    annual_ec_data = save_ec_data(year)
    save_path = f'D:/EC_data/{year}_ec_data.parquet'
    annual_ec_data.to_parquet(save_path, index=False)

Loading library list...
Done
Fetching ECs from 2024-01 ...
Doing fetch 
    select companyid, keydevid, transcriptid, headline, keydeveventtypename, 
                          transcriptcollectiontypeid, transcriptcreationdate_utc from ciq.wrds_transcript_detail 
    where mostimportantdateutc between '2024-01-01' and '2024-01-31'
    
	Found 1,040 calls
	Rows in month: 3,267
	Rows in month: 1,038
Selecting components from 1,038 transcripts
Loading library list...
Done
Fetching ECs from 2024-02 ...
Doing fetch 
    select companyid, keydevid, transcriptid, headline, keydeveventtypename, 
                          transcriptcollectiontypeid, transcriptcreationdate_utc from ciq.wrds_transcript_detail 
    where mostimportantdateutc between '2024-02-01' and '2024-02-29'
    
	Found 4,534 calls
	Rows in month: 13,342
	Rows in month: 4,526
Selecting components from 4,526 transcripts
Loading library list...
Done
Fetching ECs from 2024-03 ...
Doing fetch 
    select companyid, keydevid, trans